# Display MIPI input on DisplayPort output
---

## Aim

* This notebook illustrates how to setup a MIPI camera (Digilent Pcam 5C) and route what is seen by the camera to the DisplayPort display. 

## References
* [Digilent Pcam 5C Camera](https://reference.digilentinc.com/reference/add-ons/pcam-5c/start)    

## Revision History

* Initial Release
* 1 August 2025 | Use updated API

---

## Load _base_ Overlay, import video and image libraries

In [ ]:
from pynq.overlays.base import BaseOverlay
from pynq.devicetree import DeviceTreeSegment
from pynq.lib.video import *
import PIL.Image

base = BaseOverlay("base.bit")


## Initialize and setup the MIPI camera
Grab a handler to the MIPI hierarchy, this will initialize the camera

In [ ]:
mipi = base.mipi

Setup the camera for 1280x720 mode with 24-bits pixel. Start reading from the camera.

In [ ]:
videomode = VideoMode(1280, 720, 24)

mipi.configure(videomode)

mipi.start()

Grab a frame and display in the notebook. 

Note that the channels are arranged differently that what PIL expects, for that reason we reorder them 

In [ ]:
frame = mipi.readframe()
PIL.Image.fromarray(frame[:,:,[2,1,0]])

## Enable and start the DisplayPort output

Since it takes about 2 seconds for the hardware to handshake with the connected DisplayPort display after turning it on wait for that long

In [ ]:
displayport = DisplayPort()
displayport.configure(VideoMode(1280, 720, 24), PIXEL_RGB)

Read and display 200 frames

In [ ]:
import time
num_frames = 200
start = time.time()

for _ in range (num_frames):
    frame = displayport.newframe()
    frame[:] = mipi.readframe()
    displayport.writeframe(frame)

end = time.time()
duration = end - start
print(f"Took {duration} seconds at {num_frames / duration} FPS")

## Cleanup DisplayPort out and MIPI camera buffer

In [ ]:
displayport.close()

mipi.stop()


Copyright (c) 2022 Xilinx, Inc 
<br>
Copyright (C) 2022-2025 Advanced Micro Devices, Inc. 
<br>
SPDX-License-Identifier: BSD-3-Clause 

---

---